# Google Play Store Apps

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",100)

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error,r2_score

from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("googleplaystore.csv") #https://www.kaggle.com/datasets/lava18/google-play-store-apps?select=googleplaystore.csv

### EDA

In [3]:
df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [4]:
df.isnull().sum() #We examine the empty lines.

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


### Feature Engineering

In [6]:
for column in df.select_dtypes(include=np.object):
    df[column].fillna(df[column].mode()[0],inplace=True)
df=df.dropna()

In [7]:
df=df.reset_index()
del df["index"]

In [16]:
df["Price"].value_counts()

0.00      7150
0.99       106
2.99       101
4.99        63
1.99        53
          ... 
6.49         1
1.29         1
299.99       1
379.99       1
1.20         1
Name: Price, Length: 68, dtype: int64

In [9]:
df=df[df['Price']!="Everyone"] #We delete the Everyone values in Price.
df["Price"]=df.Price.str.replace('$','') #We delete the $ values in Price.
df['Price']=df['Price'].astype(float) ##We change the type of time to float.

In [10]:
df["Size"].value_counts()

Varies with device    1637
14M                    166
12M                    161
11M                    160
15M                    159
                      ... 
383k                     1
454k                     1
812k                     1
442k                     1
619k                     1
Name: Size, Length: 414, dtype: int64

In [11]:
df=df[df['Size']!="Varies with device"]
df["Size"]=df.Size.str.replace('.','')
df["Size"]=df.Size.str.replace('M','000000')
df["Size"]=df.Size.str.replace('k','000')
df['Size']=df['Size'].astype(int)

In [12]:
df['Reviews']=df['Reviews'].astype(int)

In [13]:
df["Installs"]=df.Installs.str.replace(',','')
df["Installs"]=df.Installs.str.replace('+','')
df['Installs']=df['Installs'].astype(int)

In [18]:
df["Last Updated"]=pd.to_datetime(df["Last Updated"])
df["day"]=(df["Last Updated"]).dt.day
df["month"]=(df["Last Updated"]).dt.month
df["year"]=(df["Last Updated"]).dt.year
df=df.drop(["Last Updated","Genres","Type"],axis=1)
#We divide date into 3 as day, month and year.

In [27]:
df

,App,Rating,Reviews,Size,Installs,Price,Content Rating,Genres,Current Ver,Android Ver,day,month,year
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,19000000,10000,0.0,0,Art & Design,1.0.0,4.0.3 and up,7,1,2018
1,Coloring book moana,3.9,967,14000000,500000,0.0,0,Art & Design;Pretend Play,2.0.0,4.0.3 and up,15,1,2018
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,87000000,5000000,0.0,0,Art & Design,1.2.4,4.0.3 and up,1,8,2018
3,Sketch - Draw & Paint,4.5,215644,25000000,50000000,0.0,1,Art & Design,Varies with device,4.2 and up,8,6,2018
4,Pixel Draw - Number Art Coloring Book,4.3,967,28000000,100000,0.0,0,Art & Design;Creativity,1.1,4.4 and up,20,6,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9361,Chemin (fr),4.8,44,619000,1000,0.0,0,Books & Reference,0.8,2.2 and up,23,3,2014
9362,FR Calculator,4.0,7,26000000,500,0.0,0,Education,1.0.0,4.1 and up,18,6,2017
9363,Sya9a Maroc - FR,4.5,38,53000000,5000,0.0,0,Education,1.48,4.1 and up,25,7,2017
9364,Fr. Mike Schmitz Audio Teachings,5.0,4,36000000,100,0.0,0,Education,1.0,4.1 and up,6,7,2018


In [20]:
df["Content Rating"].value_counts()

Everyone           6172
Teen                868
Mature 17+          368
Everyone 10+        318
Adults only 18+       2
Unrated               1
Name: Content Rating, dtype: int64

In [21]:
df=df[df['Content Rating']!="Adults only 18+"]
df=df[df['Content Rating']!="Unrated"]
df["Content Rating"]=df["Content Rating"].map({"Everyone":0,"Teen":1,"Mature 17+":2,"Everyone 10+":3})
#We replace the values in Content Rating with 0, 1, 2 and 3.
df['Content Rating']=df['Content Rating'].astype(int)

In [43]:
abs(df.corr()["Installs"].sort_values(ascending=False)) #We look at their correlations.

Installs          1.000000
Reviews           0.626191
Content Rating    0.082591
year              0.079122
Size              0.064062
Rating            0.052733
month             0.030653
Price             0.010851
day               0.047402
Name: Installs, dtype: float64

### Regression

In [44]:
x,y=df.drop(["App","Installs"],axis=1),df[["Installs"]]
x.shape

(7726, 11)

In [45]:
x=pd.get_dummies(x,drop_first=True)
x=scaler.fit_transform(x)

In [46]:
def algo_test(x,y):
    L = LinearRegression()
    E = ElasticNet()
    R = Ridge()
    Lass = Lasso()
    ETR=ExtraTreeRegressor()
    GBR=GradientBoostingRegressor()
    XGBC= XGBRegressor()
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=13)
    algos = [L,E,R,Lass,ETR,GBR,XGBC]
    algo_names = ['Linear','ElasticNet','Ridge','Lasso','Extra Tree','Gradient Boosting','XGradientBooting']
    r_squared = []
    rmse = []
    mae = []
    result = pd.DataFrame(columns = ['R_Squared','RMSE','MAE'],index = algo_names)
    for algo in algos:
        algo.fit(x_train,y_train)    
        r_squared.append(r2_score(y_test,algo.predict(x_test)))
        rmse.append(mean_squared_error(y_test, algo.predict(x_test))**.5)
        mae.append(mean_absolute_error(y_test, algo.predict(x_test)))
    result.R_Squared = r_squared
    result.RMSE = rmse
    result.MAE= mae
    return result.sort_values('R_Squared', ascending=False)

In [47]:
algo_test(x,y)

,R_Squared,RMSE,MAE
Extra Tree,9.525541e-01,8.168480e+06,1.535661e+06
Gradient Boosting,9.335276e-01,9.668569e+06,2.799454e+06
XGradientBooting,9.331202e-01,9.698158e+06,2.278748e+06
Lasso,8.283104e-01,1.553866e+07,4.939394e+06
Ridge,7.902373e-01,1.717536e+07,6.021585e+06
ElasticNet,1.841578e-02,3.715399e+07,1.214769e+07
Linear,-2.016661e+23,1.684062e+19,4.030568e+18


In [48]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [61]:
model=Sequential()
model.add(Dense(132,activation="relu"))
model.add(Dense(132,activation="relu"))
model.add(Dense(132,activation="relu"))
model.add(Dense(132,activation="relu"))
model.add(Dense(132,activation="relu"))
model.add(Dense(1))
model.compile(optimizer="adam",loss="mse")

In [62]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=128,epochs=150)
model.summary()

Epoch 1/150
49/49 [==============================] - 1s 8ms/step - loss: 2416264043560960.0000 - val_loss: 3261455088484352.0000
Epoch 2/150
49/49 [==============================] - 0s 6ms/step - loss: 2415219829637120.0000 - val_loss: 3255297179123712.0000
Epoch 3/150
49/49 [==============================] - 0s 6ms/step - loss: 2387696337027072.0000 - val_loss: 3169670026756096.0000
Epoch 4/150
49/49 [==============================] - 0s 6ms/step - loss: 2326911275499520.0000 - val_loss: 3122859446632448.0000
Epoch 5/150
49/49 [==============================] - 0s 6ms/step - loss: 2303595777097728.0000 - val_loss: 3092922719272960.0000
Epoch 6/150
49/49 [==============================] - 0s 6ms/step - loss: 2275916591923200.0000 - val_loss: 3052340915470336.0000
Epoch 7/150
49/49 [==============================] - 0s 6ms/step - loss: 2234877508321280.0000 - val_loss: 2988015995584512.0000
Epoch 8/150
49/49 [==============================] - 0s 6ms/step - loss: 2172969312845824.0000 - 

49/49 [==============================] - 0s 5ms/step - loss: 33556816461824.0000 - val_loss: 459497713696768.0000
Epoch 66/150
49/49 [==============================] - 0s 6ms/step - loss: 32383155830784.0000 - val_loss: 466879353192448.0000
Epoch 67/150
49/49 [==============================] - 0s 6ms/step - loss: 31421863297024.0000 - val_loss: 473533331275776.0000
Epoch 68/150
49/49 [==============================] - 0s 6ms/step - loss: 30637077561344.0000 - val_loss: 455281062445056.0000
Epoch 69/150
49/49 [==============================] - 0s 6ms/step - loss: 29577829154816.0000 - val_loss: 471549089939456.0000
Epoch 70/150
49/49 [==============================] - 0s 6ms/step - loss: 28712510685184.0000 - val_loss: 448157523640320.0000
Epoch 71/150
49/49 [==============================] - 0s 6ms/step - loss: 28041625468928.0000 - val_loss: 459852149161984.0000
Epoch 72/150
49/49 [==============================] - 0s 6ms/step - loss: 27119610494976.0000 - val_loss: 438015327469568.00

In [63]:
tahmin=model.predict(x_test)

49/49 [==============================] - 0s 1ms/step


In [64]:
r2_score(tahmin,y_test)

0.8926532002699549

In [65]:
(mean_squared_error(tahmin,y_test))**0.5

18785511.10371431